## Zero Arb On chain Study 

This study compares on-chain settlement prices of COW protocol vs UNIv3. 

It filters all the trades on both sides that have the same block, same buy token and same sell token. 

In particular, it compares the cow price with 1)the best available Univ3 price in the block called `univ3BlockMin` and 2) the volume weighted average price of univ3 in the block, the `univ3VWAP` price. 

The best available price given by `univ3BlockMin` represents the best price univ3 executed at but does not take into consideration the volumes of these trades. A higher volume trade is expected to fetch a worse price given the impact to the poolso its not fair to compare the price of a small trade with a price of a high volume trade. To adjust for this, the vwap price of the block is constructed using the UNiv3 trades and allows one to compare the price obtained by cow to the volume weighted price of the block.  

This study only looks at Univ3 as proof of concept of methodology, but once peer reviewed, can be easily extended to include all major DEXs and aggregators that settle on chain, and extending the dataset. The hit rate of trades that match is roughly 2.7% This means that going over 5,000 COW trades, 136 trades happen to have corresponding UNIv3 trades of the same tokens in the same block.Adding more DEXs is expected to improve this, and would enrich the dataset. To our knowledge, no such study for COW protocol pricing comparison exists. We believe the COW ecosystem would benefit from an extention of this study to include other DEXs and fetching more laborious details such as UNI gas adjustments. 

The sources of data used are the COW and UNIv3 subgraph. Where richer data does exist under paid services, we wish to keep the study open source for anyone to run, play around with and verify. 


Notes on the study:

- If f a block has one univ3 trade, then `univ3BlockMax` = `univ3vwap` 
- COW prices retrieves already include all fees including gas prices. So to compare fairly, we adjust the UNIv3 price to reflect the gas fees as well. However, difficulty in retrieving exact data from open source data, meant that some heurestics were used. Details are explained in relevent cells. 


Outline: 

1. COW Trades : Obtain the last COW trades for a given query size.
2. UNI Swaps: Obtain UNI swaps that correspond to the same period as the COW dataset. 
3. Adjust UNI prices with gas fees. 
3. Merge COW with UNI on timestamp, token1, token2 : This results in the trades that are overlapping only. 
4. Group UNI trades together and derive one price per block. Once deriving the one price to be the Block Max price and second the VWAP. 
5. Derive stats



Internal notes: 

- fix variable names 
- add volume stuff 

### 1. COW Trades

In [299]:
from datastreams.datastream import Streamer

import matplotlib.pyplot as plt
import pandas as pd
import polars as pl

# These commands enlarge the column size of the dataframe so things like 0x... are not truncated
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [300]:
# instantiate Streamer class. Note that we need two separate streamer classes, otherwise the queries will be overwritten. 
cow_ds = Streamer('https://api.thegraph.com/subgraphs/name/cowprotocol/cow')
#cow_ds2 = Streamer('https://api.thegraph.com/subgraphs/name/cowprotocol/cow')

In [325]:
# DEFINE TIMESTAMP HERE. Timstamp is used for replication quality assurance purposes. 
timestamp =  1672574400

# we set a fixed query size number. The Cow settlements and Uniswap swaps query are multiples larger than this initial query size.
query_size = 100

#Filter size - We filter trades out that are smaller than $1000 USD size
filter_usd = 100

In [326]:
# query COW schema: trades
trades_fp = cow_ds.queryDict.get('trades')

# trades query path that gets token a -> token b trades
trades_qp = trades_fp(
    first=query_size,
    orderBy='timestamp',
    orderDirection='desc',
    where = {
    'timestamp_gt': timestamp,
    'buyAmountUsd_gt': filter_usd
    }
)

# run query
trades_df = cow_ds.runQuery(trades_qp)

FIELD - trades


In [327]:
print(f'query returned {len(trades_df)} rows')

query returned 100 rows


In [328]:
# get unique values in trades_df column to verify the query results.
len(trades_df['trades_buyToken_id'].unique())

31

In [329]:
trades_df.dtypes

trades_id                object
trades_timestamp          int64
trades_gasPrice           int64
trades_feeAmount         object
trades_txHash            object
trades_settlement_id     object
trades_buyAmount         object
trades_sellAmount        object
trades_sellToken_id      object
trades_buyToken_id       object
trades_order_id          object
trades_buyAmountEth     float64
trades_sellAmountEth    float64
trades_buyAmountUsd     float64
trades_sellAmountUsd    float64
endpoint                 object
dtype: object

In [330]:
trades_df.head(20)

,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_buyAmount,trades_sellAmount,trades_sellToken_id,trades_buyToken_id,trades_order_id,trades_buyAmountEth,trades_sellAmountEth,trades_buyAmountUsd,trades_sellAmountUsd,endpoint
0,0x66083f89ab1511b24ff67493ce716dfd2d04867d71e020c171a8069fed28b6ff0c9b4b81dacc6828591afd98a01823a5873b6b15640b2f54|0xcd28189df3f765324dc8b4800dc902452220690e5ae9f6950ea9926dded0ee83|153,1678452851,25074094968,9917255,0xcd28189df3f765324dc8b4800dc902452220690e5ae9f6950ea9926dded0ee83,0xcd28189df3f765324dc8b4800dc902452220690e5ae9f6950ea9926dded0ee83,38097401543449055064668,2000000000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xd04e772bc0d591fbd288f2e2a86afa3d3cb647f8,0x66083f89ab1511b24ff67493ce716dfd2d04867d71e020c171a8069fed28b6ff0c9b4b81dacc6828591afd98a01823a5873b6b15640b2f54,1.418160,1.434532,1977.174748,2000.000000,cow
1,0xc88ec2b2c7524341f2d1c985f7d6f117e6e297e103963cae8cdb688feaf3c93e93aa01eab7f9d6c8511a4a873fea19073334c004640b2ec6|0x2734bf4a97cb38d194c539c648e350d329667caa0a126ad4fbda58448fc078c1|140,1678452743,25444490559,5768040,0x2734bf4a97cb38d194c539c648e350d329667caa0a126ad4fbda58448fc078c1,0x2734bf4a97cb38d194c539c648e350d329667caa0a126ad4fbda58448fc078c1,2147669204675768309,3000000000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0xc88ec2b2c7524341f2d1c985f7d6f117e6e297e103963cae8cdb688feaf3c93e93aa01eab7f9d6c8511a4a873fea19073334c004640b2ec6,2.147669,2.151798,2994.243464,3000.000000,cow
2,0xd7cccada9ccc97256d3151b53b40ebf01e5532444c531d5a38f62a1d67ffa2fe0c9b4b81dacc6828591afd98a01823a5873b6b15640b2e7e|0xd478ceb97aee884f4015ecedf3e7936c38a1d9946053ebf52d60afb615e13135|142,1678452707,26562222091,10029197,0xd478ceb97aee884f4015ecedf3e7936c38a1d9946053ebf52d60afb615e13135,0xd478ceb97aee884f4015ecedf3e7936c38a1d9946053ebf52d60afb615e13135,18125350760090508839833,1000000000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xd04e772bc0d591fbd288f2e2a86afa3d3cb647f8,0xd7cccada9ccc97256d3151b53b40ebf01e5532444c531d5a38f62a1d67ffa2fe0c9b4b81dacc6828591afd98a01823a5873b6b15640b2e7e,0.698534,0.717266,973.884725,1000.000000,cow
3,0xaceeddc91017e29169acc1b42dcc8d64dc0f415f953c7ca90d6ad34b51f188e66e869cadc1cb3d4c6291e6e939b5b55d51c69084640b2ea2|0xd478ceb97aee884f4015ecedf3e7936c38a1d9946053ebf52d60afb615e13135|142,1678452707,26562222091,5356732995621110784,0xd478ceb97aee884f4015ecedf3e7936c38a1d9946053ebf52d60afb615e13135,0xd478ceb97aee884f4015ecedf3e7936c38a1d9946053ebf52d60afb615e13135,1817272269,697370207447521938768,0x1e4746dc744503b53b4a082cb3607b169a289090,0xdac17f958d2ee523a2206206994597c13d831ec7,0xaceeddc91017e29169acc1b42dcc8d64dc0f415f953c7ca90d6ad34b51f188e66e869cadc1cb3d4c6291e6e939b5b55d51c69084640b2ea2,1.303468,1.320736,1817.272269,1841.319096,cow
4,0x28d7d3bf4ecec9520590fc33318721513efea97e6e7c7f1c078e6fe6ef8fc6ebcd9cfed808c0ba09e153eb6ab5ba3076181d1ace640b2dd3|0x5b30641f806df0f4bbe9927410a7b49c08f41776887ee90014eeba4cd49dcd23|98,1678452563,25871791575,4860402,0x5b30641f806df0f4bbe9927410a7b49c08f41776887ee90014eeba4cd49dcd23,0x5b30641f806df0f4bbe9927410a7b49c08f41776887ee90014eeba4cd49dcd23,12530406922076655921,17500000000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0x28d7d3bf4ecec9520590fc33318721513efea97e6e7c7f1c078e6fe6ef8fc6ebcd9cfed808c0ba09e153eb6ab5ba3076181d1ace640b2dd3,12.530407,12.552352,17469.405687,17500.000000,cow
5,0x8878be27c10bd3ce9e204b44142dca88a80951289dae843f92c123f0e26aed4d77fb4fa1aba92576942ad34bc47834059b84e693640b2dcd|0x69f9d311b4c440339f7fdf76a9bd36cbe9fc20d5fd10dd69dc41f33d26fbafca|103,1678452479,23993646687,10272379,0x69f9d311b4c440339f7fdf76a9bd36cbe9fc20d5fd10dd69dc41f33d26fbafca,0x69f9d311b4c440339f7fdf76a9bd36cbe9fc20d5fd10dd69dc41f33d26fbafca,709640022407011338,1000000000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0x8878be27c10bd3ce9e204b44142dca88a80951289dae843f92c123f0e26aed4d77fb4

In [331]:
# Set query size large enough to retrieve all tokens in the graph
query_size_token = 10000000

token_fp = cow_ds.queryDict.get('tokens')

# add parameters to the settlements_qp.
token_qp = token_fp(
    first=query_size_token,
)

# run query
token_df = cow_ds.runQuery(token_qp)

FIELD - tokens


In [332]:
# Merge1 on sell token address.  x = sell token 

merged_df = pd.merge(trades_df, token_df, left_on='trades_sellToken_id', right_on='tokens_address')

In [333]:
# Merge2 on buy token address.  y = buy token 
merged_df2 = pd.merge(merged_df, token_df, left_on='trades_buyToken_id', right_on='tokens_address')

In [334]:
merged_df2 = merged_df2.rename(columns={
    "tokens_symbol_x": "sell_token_symbol",
    "tokens_symbol_y":"buy_token_symbol",
    "tokens_decimals_x": "sell_token_decimal", 
    "tokens_decimals_y": "buy_token_decimal"})

In [335]:
# filter necessary columns
cow_df = merged_df2[[
    'trades_id',
    'trades_timestamp', 
    'trades_gasPrice', 
    'trades_feeAmount',                 
    'trades_txHash',                    
    'trades_settlement_id',   
    'trades_sellAmount',
    'sell_token_decimal',
    'trades_buyAmount',   
    'buy_token_decimal',
    'trades_sellToken_id',              
    'trades_buyToken_id',               
    'trades_order_id',                  
    'sell_token_symbol',
    'buy_token_symbol',
    'trades_sellAmountUsd'
    ]]


In [336]:
cow_df.loc[:, 'sell_amount'] = cow_df['trades_sellAmount'] / (10**cow_df['sell_token_decimal'])
cow_df.loc[:, 'buy_amount'] = cow_df['trades_buyAmount'] / (10**cow_df['buy_token_decimal'])


/tmp/ipykernel_37253/1103516907.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cow_df.loc[:, 'sell_amount'] = cow_df['trades_sellAmount'] / (10**cow_df['sell_token_decimal'])
/tmp/ipykernel_37253/1103516907.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cow_df.loc[:, 'buy_amount'] = cow_df['trades_buyAmount'] / (10**cow_df['buy_token_decimal'])


In [337]:
# calculate COW price defined sell amount / buy amount
cow_df['cow_sell_buy_ratio'] = cow_df['sell_amount'] / cow_df['buy_amount']

/tmp/ipykernel_37253/3636551783.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cow_df['cow_sell_buy_ratio'] = cow_df['sell_amount'] / cow_df['buy_amount']


In [338]:
cow_df.shape

(100, 19)

In [339]:
cow_df.dtypes

trades_id                object
trades_timestamp          int64
trades_gasPrice           int64
trades_feeAmount         object
trades_txHash            object
trades_settlement_id     object
trades_sellAmount        object
sell_token_decimal        int64
trades_buyAmount         object
buy_token_decimal         int64
trades_sellToken_id      object
trades_buyToken_id       object
trades_order_id          object
sell_token_symbol        object
buy_token_symbol         object
trades_sellAmountUsd    float64
sell_amount              object
buy_amount               object
cow_sell_buy_ratio       object
dtype: object

In [340]:
cow_df.head(5)

,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_sellAmount,sell_token_decimal,trades_buyAmount,buy_token_decimal,trades_sellToken_id,trades_buyToken_id,trades_order_id,sell_token_symbol,buy_token_symbol,trades_sellAmountUsd,sell_amount,buy_amount,cow_sell_buy_ratio
0,0x66083f89ab1511b24ff67493ce716dfd2d04867d71e020c171a8069fed28b6ff0c9b4b81dacc6828591afd98a01823a5873b6b15640b2f54|0xcd28189df3f765324dc8b4800dc902452220690e5ae9f6950ea9926dded0ee83|153,1678452851,25074094968,9917255,0xcd28189df3f765324dc8b4800dc902452220690e5ae9f6950ea9926dded0ee83,0xcd28189df3f765324dc8b4800dc902452220690e5ae9f6950ea9926dded0ee83,2000000000,6,38097401543449055064668,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xd04e772bc0d591fbd288f2e2a86afa3d3cb647f8,0x66083f89ab1511b24ff67493ce716dfd2d04867d71e020c171a8069fed28b6ff0c9b4b81dacc6828591afd98a01823a5873b6b15640b2f54,USDC,GPT,2000.0,2000.0,38097.401543,0.052497
1,0xd7cccada9ccc97256d3151b53b40ebf01e5532444c531d5a38f62a1d67ffa2fe0c9b4b81dacc6828591afd98a01823a5873b6b15640b2e7e|0xd478ceb97aee884f4015ecedf3e7936c38a1d9946053ebf52d60afb615e13135|142,1678452707,26562222091,10029197,0xd478ceb97aee884f4015ecedf3e7936c38a1d9946053ebf52d60afb615e13135,0xd478ceb97aee884f4015ecedf3e7936c38a1d9946053ebf52d60afb615e13135,1000000000,6,18125350760090508839833,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xd04e772bc0d591fbd288f2e2a86afa3d3cb647f8,0xd7cccada9ccc97256d3151b53b40ebf01e5532444c531d5a38f62a1d67ffa2fe0c9b4b81dacc6828591afd98a01823a5873b6b15640b2e7e,USDC,GPT,1000.0,1000.0,18125.35076,0.055171
2,0xc88ec2b2c7524341f2d1c985f7d6f117e6e297e103963cae8cdb688feaf3c93e93aa01eab7f9d6c8511a4a873fea19073334c004640b2ec6|0x2734bf4a97cb38d194c539c648e350d329667caa0a126ad4fbda58448fc078c1|140,1678452743,25444490559,5768040,0x2734bf4a97cb38d194c539c648e350d329667caa0a126ad4fbda58448fc078c1,0x2734bf4a97cb38d194c539c648e350d329667caa0a126ad4fbda58448fc078c1,3000000000,6,2147669204675768309,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0xc88ec2b2c7524341f2d1c985f7d6f117e6e297e103963cae8cdb688feaf3c93e93aa01eab7f9d6c8511a4a873fea19073334c004640b2ec6,USDC,,3000.0,3000.0,2.147669,1396.863164
3,0x28d7d3bf4ecec9520590fc33318721513efea97e6e7c7f1c078e6fe6ef8fc6ebcd9cfed808c0ba09e153eb6ab5ba3076181d1ace640b2dd3|0x5b30641f806df0f4bbe9927410a7b49c08f41776887ee90014eeba4cd49dcd23|98,1678452563,25871791575,4860402,0x5b30641f806df0f4bbe9927410a7b49c08f41776887ee90014eeba4cd49dcd23,0x5b30641f806df0f4bbe9927410a7b49c08f41776887ee90014eeba4cd49dcd23,17500000000,6,12530406922076655921,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0x28d7d3bf4ecec9520590fc33318721513efea97e6e7c7f1c078e6fe6ef8fc6ebcd9cfed808c0ba09e153eb6ab5ba3076181d1ace640b2dd3,USDC,,17500.0,17500.0,12.530407,1396.602689
4,0x8878be27c10bd3ce9e204b44142dca88a80951289dae843f92c123f0e26aed4d77fb4fa1aba92576942ad34bc47834059b84e693640b2dcd|0x69f9d311b4c440339f7fdf76a9bd36cbe9fc20d5fd10dd69dc41f33d26fbafca|103,1678452479,23993646687,10272379,0x69f9d311b4c440339f7fdf76a9bd36cbe9fc20d5fd10dd69dc41f33d26fbafca,0x69f9d311b4c440339f7fdf76a9bd36cbe9fc20d5fd10dd69dc41f33d26fbafca,1000000000,6,709640022407011338,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee,0x8878be27c10bd3ce9e204b44142dca88a80951289dae843f92c123f0e26aed4d77fb4fa1aba92576942ad34bc47834059b84e693640b2dcd,USDC,,1000.0,1000.0,0.70964,1409.165166


In [341]:
# check first and last timestamps

first_timestamp_cow = cow_df['trades_timestamp'].min()
last_timestamp_cow = cow_df['trades_timestamp'].max()

print("First timestamp:", first_timestamp_cow)
print("Last timestamp:", last_timestamp_cow)

First timestamp: 1678446359
Last timestamp: 1678452851


In [342]:
import time

# Convert Unix timestamps to human-readable dates
first_date_cow = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(first_timestamp_cow))
last_date_cow = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(last_timestamp_cow))

print("First date:", first_date_cow)
print("Last date:", last_date_cow)

First date: 2023-03-10 11:05:59
Last date: 2023-03-10 12:54:11


In [343]:
timestamps_list = cow_df['trades_timestamp'].to_list()
# get unique values in cow_timestamps
cow_timestamps = list(set(timestamps_list))

In [344]:
len(cow_timestamps)

79

### 2. Uniswap V3 Swaps

In [345]:
# instantiate Streamer object. |
# Note - unlike the cow queries, univ3 does not require multiple streamer instantations because the swaps field path is reset each iteration. 
# If the Cow queries were updated to use the same method, we could use the same streamer object for all queries.
univ3_ds = Streamer('https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum')

In [346]:
# get a query field path from the query dictionary which is automatically populated in the Streamer object
uni_swaps_fp = univ3_ds.queryDict.get('swaps')

In [347]:
swaps_df_list = []

In [348]:
for i in range(0, len(cow_timestamps)): # inner loop through cowswap timestamps
    print(f'round {i} of {len(cow_timestamps)}, timestamp: {cow_timestamps[i]}')
    # define the query path
    uni_swaps_qp = uni_swaps_fp(
        first=100,
        orderBy='timestamp',
        orderDirection='desc',
        where = {'timestamp': cow_timestamps[i], 'amountInUSD_gt': filter_usd, 'amountOutUSD_gt': filter_usd} 
        )

    # run uni swaps query
    df = univ3_ds.runQuery(uni_swaps_qp)

    if df.empty:
        print('uni swaps empty')
        continue
    else:
        swaps_df_list.append(df)


round 0 of 79, timestamp: 1678450691
FIELD - swaps
round 1 of 79, timestamp: 1678452743
FIELD - swaps
round 2 of 79, timestamp: 1678448135
FIELD - swaps
round 3 of 79, timestamp: 1678448675
FIELD - swaps
round 4 of 79, timestamp: 1678450727
FIELD - swaps
round 5 of 79, timestamp: 1678447655
FIELD - swaps
round 6 of 79, timestamp: 1678448171
FIELD - swaps
round 7 of 79, timestamp: 1678451783
FIELD - swaps
round 8 of 79, timestamp: 1678449227
FIELD - swaps
round 9 of 79, timestamp: 1678446671
FIELD - swaps
round 10 of 79, timestamp: 1678448207
FIELD - swaps
round 11 of 79, timestamp: 1678447187
FIELD - swaps
round 12 of 79, timestamp: 1678450787
FIELD - swaps
round 13 of 79, timestamp: 1678449263
FIELD - swaps
round 14 of 79, timestamp: 1678452851
FIELD - swaps
round 15 of 79, timestamp: 1678448243
FIELD - swaps
round 16 of 79, timestamp: 1678448771
FIELD - swaps
round 17 of 79, timestamp: 1678450307
FIELD - swaps
round 18 of 79, timestamp: 1678452359
FIELD - swaps
round 19 of 79, timest

In [349]:
uni_swaps_df = pd.concat(swaps_df_list)

In [350]:
uni_swaps_df.shape

(668, 20)

In [351]:
uni_swaps_df.dtypes

swaps_id               object
swaps_hash             object
swaps_nonce             int64
swaps_logIndex          int64
swaps_gasLimit          int64
swaps_gasUsed          object
swaps_gasPrice          int64
swaps_protocol_id      object
swaps_account_id       object
swaps_pool_id          object
swaps_blockNumber       int64
swaps_timestamp         int64
swaps_tick              int64
swaps_tokenIn_id       object
swaps_amountIn         object
swaps_amountInUSD     float64
swaps_tokenOut_id      object
swaps_amountOut        object
swaps_amountOutUSD    float64
endpoint               object
dtype: object

In [352]:
uni_swaps_df.head(5)

,swaps_id,swaps_hash,swaps_nonce,swaps_logIndex,swaps_gasLimit,swaps_gasUsed,swaps_gasPrice,swaps_protocol_id,swaps_account_id,swaps_pool_id,swaps_blockNumber,swaps_timestamp,swaps_tick,swaps_tokenIn_id,swaps_amountIn,swaps_amountInUSD,swaps_tokenOut_id,swaps_amountOut,swaps_amountOutUSD,endpoint
0,0xe4fab4a0084618b492ea1c6f3cea09c7f74b63a297849af91b35b5ba481037eb37000000,0xe4fab4a0084618b492ea1c6f3cea09c7f74b63a297849af91b35b5ba481037eb,1019,55,350770,None,23075521357,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x826ca8bbed18c587320d8b822f8c632544271e99,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,16797679,1678450691,203979,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,2823591691214060678,3906.078150,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,3911206227,3911.206227,uniswap-v3-ethereum
1,0x9a6785460a99bb41133fdd209b3c67ef736337bc2fafc8d87d502f52a8fc442c30000000,0x9a6785460a99bb41133fdd209b3c67ef736337bc2fafc8d87d502f52a8fc442c,127195,48,260382,None,22996469245,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x4603180bbb8221157880afaa84638e0fc467738d,0x92ab871abb9d567aa276b2ce58d0203d84e0181e,16797679,1678450691,-283823,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,4799063697,4799.063697,0x5283d291dbcf85356a21ba090e6db59121208b44,10156885845737085399878,4791.159195,uniswap-v3-ethereum
2,0x4bd75590e8411327738e928bddff495a087377509016fef8763df79e29faa7619e000000,0x4bd75590e8411327738e928bddff495a087377509016fef8763df79e29faa761,98,158,298665,None,23075521357,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x9a4193a8738808d65da31ee40cfd1d7fc1dd298b,0x68d01efdfbe5c0eebe3b19b571b18ae6d6ee16fc,16797679,1678450691,-73202,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,4280661385101695258,5921.747806,0x3446dd70b2d52a6bf4a5a192d9b0a161295ab7f9,6414890468310065275438,5847.484526,uniswap-v3-ethereum
3,0x4bd75590e8411327738e928bddff495a087377509016fef8763df79e29faa7619b000000,0x4bd75590e8411327738e928bddff495a087377509016fef8763df79e29faa761,98,155,298665,None,23075521357,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x9a4193a8738808d65da31ee40cfd1d7fc1dd298b,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,16797679,1678450691,203979,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,5935480721,5935.480721,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,4280661385101695258,5921.747806,uniswap-v3-ethereum
4,0x4888e1ebdb74dbc93895439763cfb2322cfc5e6a9a2dbbd704406c4389ad11e72d000000,0x4888e1ebdb74dbc93895439763cfb2322cfc5e6a9a2dbbd704406c4389ad11e7,165592,45,327938,None,23237437847,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x042523db4f3effc33d2742022b2490258494f8b3,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,16797679,1678450691,203979,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,231152915632,231152.915632,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,166785577727074028193,230726.525676,uniswap-v3-ethereum


In [353]:
# check first and last timestamps

first_timestamp = uni_swaps_df['swaps_timestamp'].min()
last_timestamp = uni_swaps_df['swaps_timestamp'].max()

print("First timestamp:", first_timestamp)
print("Last timestamp:", last_timestamp)

First timestamp: 1678446359
Last timestamp: 1678452851


In [355]:
import time

# Convert Unix timestamps to human-readable dates
first_date = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(first_timestamp))
last_date = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(last_timestamp))

print("First date:", first_date)
print("Last date:", last_date)


First date: 2023-03-10 11:05:59
Last date: 2023-03-10 12:54:11


In [356]:
# Set query size large enough to retrieve all tokens in the graph
query_size_token = 10000

uni_token_fp = univ3_ds.queryDict.get('tokens')

# add parameters to the settlements_qp.
uni_token_qp = uni_token_fp(
    first=query_size_token,
)

# run query
uni_token_df = univ3_ds.runQuery(uni_token_qp)

FIELD - tokens


In [357]:
uni_token_df.shape

(7196, 7)

In [358]:
uni_token_df.dtypes

tokens_id                       object
tokens_name                     object
tokens_symbol                   object
tokens_decimals                  int64
tokens_lastPriceUSD            float64
tokens_lastPriceBlockNumber      int64
endpoint                        object
dtype: object

In [359]:
# Merge1 on tokenIN address.   x is in 

merge1_uni_df = pd.merge(uni_swaps_df, uni_token_df, left_on='swaps_tokenIn_id', right_on='tokens_id')

In [360]:
# Merge2 on tokenOUT address.  y is out 

merge2_uni_df = pd.merge(merge1_uni_df, uni_token_df, left_on='swaps_tokenOut_id', right_on='tokens_id')

In [361]:
merge2_uni_df.dtypes

swaps_id                          object
swaps_hash                        object
swaps_nonce                        int64
swaps_logIndex                     int64
swaps_gasLimit                     int64
swaps_gasUsed                     object
swaps_gasPrice                     int64
swaps_protocol_id                 object
swaps_account_id                  object
swaps_pool_id                     object
swaps_blockNumber                  int64
swaps_timestamp                    int64
swaps_tick                         int64
swaps_tokenIn_id                  object
swaps_amountIn                    object
swaps_amountInUSD                float64
swaps_tokenOut_id                 object
swaps_amountOut                   object
swaps_amountOutUSD               float64
endpoint_x                        object
tokens_id_x                       object
tokens_name_x                     object
tokens_symbol_x                   object
tokens_decimals_x                  int64
tokens_lastPrice

In [362]:
merge2_uni_df = merge2_uni_df.rename(columns={
    "tokens_symbol_x": "tokenIn_symbol",
    "tokens_symbol_y":"tokenOut_symbol",
    "tokens_decimals_x": "tokenIn_decimal", 
    "tokens_decimals_y": "tokenOut_decimal"})

In [363]:
# filter necessary columns
uni_df = merge2_uni_df[[
    'swaps_id',
    'swaps_hash',
    'swaps_protocol_id',
    'swaps_blockNumber',
    'swaps_timestamp',
    'swaps_tokenIn_id',
    'tokenIn_symbol',
    'tokenIn_decimal',
    'swaps_amountIn',
    'swaps_tokenOut_id',
    'tokenOut_symbol',
    'tokenOut_decimal',
    'swaps_amountOut',
    'swaps_gasLimit',                   
    'swaps_gasUsed',                     
    'swaps_gasPrice',                   
    'swaps_pool_id',
    'swaps_amountOutUSD' 
    ]]

In [364]:
uni_df.head(5)

,swaps_id,swaps_hash,swaps_protocol_id,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,tokenIn_symbol,tokenIn_decimal,swaps_amountIn,swaps_tokenOut_id,tokenOut_symbol,tokenOut_decimal,swaps_amountOut,swaps_gasLimit,swaps_gasUsed,swaps_gasPrice,swaps_pool_id,swaps_amountOutUSD
0,0xe4fab4a0084618b492ea1c6f3cea09c7f74b63a297849af91b35b5ba481037eb37000000,0xe4fab4a0084618b492ea1c6f3cea09c7f74b63a297849af91b35b5ba481037eb,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797679,1678450691,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,2823591691214060678,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,3911206227,350770,None,23075521357,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,3911.206227
1,0x330c4168f95efddde5fb82ffe1832991c88c1e20337ebac8e8cbdef8e36a27311f000000,0x330c4168f95efddde5fb82ffe1832991c88c1e20337ebac8e8cbdef8e36a2731,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797679,1678450691,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,128031297941806173106,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,177245294150,1825717,None,23237437847,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,177245.294150
2,0x172c4f3a11255b98b4811c496588f3ad6b21fe11556f1e0d50e4fd50b23d5a0ef0000000,0x172c4f3a11255b98b4811c496588f3ad6b21fe11556f1e0d50e4fd50b23d5a0e,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797679,1678450691,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,956535636118829721,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,1325000000,235837,None,23075521357,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,1325.000000
3,0xd1e17a0b410ff5023bfae437c2ba672938e52dcebc3279fc40cc5f9b76bedeba5f000000,0xd1e17a0b410ff5023bfae437c2ba672938e52dcebc3279fc40cc5f9b76bedeba,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797468,1678448135,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,144833161400523502,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,200000000,256347,None,24828566233,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,200.000000
4,0xdc7f74fe8ab9fda790cdf55610127f2514c362da15a231728b9a4da99ee16e8802000000,0xdc7f74fe8ab9fda790cdf55610127f2514c362da15a231728b9a4da99ee16e88,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797513,1678448675,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,57127575212311646324,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,78355027835,338535,None,26439661157,0xe0554a476a092703abdb3ef35c80e0d76d32939f,78355.027835


In [365]:
# Get amounts in a uniform decimal 
uni_df.loc[:, 'univ3_In_amount'] = uni_df['swaps_amountIn'] / (10**uni_df['tokenIn_decimal'])
uni_df.loc[:, 'univ3_Out_amount'] = uni_df['swaps_amountOut'] / (10**uni_df['tokenOut_decimal'])

/tmp/ipykernel_37253/2168625794.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_df.loc[:, 'univ3_In_amount'] = uni_df['swaps_amountIn'] / (10**uni_df['tokenIn_decimal'])
/tmp/ipykernel_37253/2168625794.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_df.loc[:, 'univ3_Out_amount'] = uni_df['swaps_amountOut'] / (10**uni_df['tokenOut_decimal'])


In [366]:
# calculate UNI swap price defined In amount / Out amount
uni_df['uni_in_out_ratio'] = uni_df['univ3_In_amount'] / uni_df['univ3_Out_amount']

/tmp/ipykernel_37253/1576826904.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_df['uni_in_out_ratio'] = uni_df['univ3_In_amount'] / uni_df['univ3_Out_amount']


In [367]:
uni_df.shape

(668, 21)

In [368]:
uni_df.head(5)

,swaps_id,swaps_hash,swaps_protocol_id,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,tokenIn_symbol,tokenIn_decimal,swaps_amountIn,swaps_tokenOut_id,tokenOut_symbol,tokenOut_decimal,swaps_amountOut,swaps_gasLimit,swaps_gasUsed,swaps_gasPrice,swaps_pool_id,swaps_amountOutUSD,univ3_In_amount,univ3_Out_amount,uni_in_out_ratio
0,0xe4fab4a0084618b492ea1c6f3cea09c7f74b63a297849af91b35b5ba481037eb37000000,0xe4fab4a0084618b492ea1c6f3cea09c7f74b63a297849af91b35b5ba481037eb,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797679,1678450691,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,2823591691214060678,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,3911206227,350770,None,23075521357,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,3911.206227,2.823592,3911.206227,0.000722
1,0x330c4168f95efddde5fb82ffe1832991c88c1e20337ebac8e8cbdef8e36a27311f000000,0x330c4168f95efddde5fb82ffe1832991c88c1e20337ebac8e8cbdef8e36a2731,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797679,1678450691,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,128031297941806173106,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,177245294150,1825717,None,23237437847,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,177245.294150,128.031298,177245.29415,0.000722
2,0x172c4f3a11255b98b4811c496588f3ad6b21fe11556f1e0d50e4fd50b23d5a0ef0000000,0x172c4f3a11255b98b4811c496588f3ad6b21fe11556f1e0d50e4fd50b23d5a0e,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797679,1678450691,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,956535636118829721,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,1325000000,235837,None,23075521357,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,1325.000000,0.956536,1325.0,0.000722
3,0xd1e17a0b410ff5023bfae437c2ba672938e52dcebc3279fc40cc5f9b76bedeba5f000000,0xd1e17a0b410ff5023bfae437c2ba672938e52dcebc3279fc40cc5f9b76bedeba,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797468,1678448135,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,144833161400523502,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,200000000,256347,None,24828566233,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,200.000000,0.144833,200.0,0.000724
4,0xdc7f74fe8ab9fda790cdf55610127f2514c362da15a231728b9a4da99ee16e8802000000,0xdc7f74fe8ab9fda790cdf55610127f2514c362da15a231728b9a4da99ee16e88,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797513,1678448675,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,57127575212311646324,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,78355027835,338535,None,26439661157,0xe0554a476a092703abdb3ef35c80e0d76d32939f,78355.027835,57.127575,78355.027835,0.000729


### 3. Adjust uni price by gas price

In [369]:
# convert gas prices to ETH. 
uni_df['gas_wei'] = uni_df['swaps_gasPrice']/10**18 

/tmp/ipykernel_37253/4059436116.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_df['gas_wei'] = uni_df['swaps_gasPrice']/10**18


In [370]:
# Calculate gas in USD terms. 1500 price of ETH used as median. Better data source needed to get for a given timestamp. 
uni_df['gasUSD'] = uni_df['gas_wei'] * uni_df['swaps_gasLimit'] *1500 

/tmp/ipykernel_37253/4630247.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_df['gasUSD'] = uni_df['gas_wei'] * uni_df['swaps_gasLimit'] *1500


In [371]:
uni_df.shape

(668, 23)

In [372]:
uni_df.head(5)

,swaps_id,swaps_hash,swaps_protocol_id,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,tokenIn_symbol,tokenIn_decimal,swaps_amountIn,swaps_tokenOut_id,tokenOut_symbol,tokenOut_decimal,swaps_amountOut,swaps_gasLimit,swaps_gasUsed,swaps_gasPrice,swaps_pool_id,swaps_amountOutUSD,univ3_In_amount,univ3_Out_amount,uni_in_out_ratio,gas_wei,gasUSD
0,0xe4fab4a0084618b492ea1c6f3cea09c7f74b63a297849af91b35b5ba481037eb37000000,0xe4fab4a0084618b492ea1c6f3cea09c7f74b63a297849af91b35b5ba481037eb,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797679,1678450691,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,2823591691214060678,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,3911206227,350770,None,23075521357,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,3911.206227,2.823592,3911.206227,0.000722,2.307552e-08,12.141301
1,0x330c4168f95efddde5fb82ffe1832991c88c1e20337ebac8e8cbdef8e36a27311f000000,0x330c4168f95efddde5fb82ffe1832991c88c1e20337ebac8e8cbdef8e36a2731,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797679,1678450691,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,128031297941806173106,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,177245294150,1825717,None,23237437847,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,177245.294150,128.031298,177245.29415,0.000722,2.323744e-08,63.637478
2,0x172c4f3a11255b98b4811c496588f3ad6b21fe11556f1e0d50e4fd50b23d5a0ef0000000,0x172c4f3a11255b98b4811c496588f3ad6b21fe11556f1e0d50e4fd50b23d5a0e,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797679,1678450691,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,956535636118829721,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,1325000000,235837,None,23075521357,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,1325.000000,0.956536,1325.0,0.000722,2.307552e-08,8.163093
3,0xd1e17a0b410ff5023bfae437c2ba672938e52dcebc3279fc40cc5f9b76bedeba5f000000,0xd1e17a0b410ff5023bfae437c2ba672938e52dcebc3279fc40cc5f9b76bedeba,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797468,1678448135,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,144833161400523502,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,200000000,256347,None,24828566233,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,200.000000,0.144833,200.0,0.000724,2.482857e-08,9.547093
4,0xdc7f74fe8ab9fda790cdf55610127f2514c362da15a231728b9a4da99ee16e8802000000,0xdc7f74fe8ab9fda790cdf55610127f2514c362da15a231728b9a4da99ee16e88,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797513,1678448675,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,57127575212311646324,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,78355027835,338535,None,26439661157,0xe0554a476a092703abdb3ef35c80e0d76d32939f,78355.027835,57.127575,78355.027835,0.000729,2.643966e-08,13.426126


In [373]:
# Use the amountOutUSD as estimator of the trade volume and calculate the percentage of gas fees of volume 
# The UNIv3 subgraph has a field called `gasUsed` but actually it is not the gas used, it represents the limit 
# Usually, trades are settling lower than this limit. Hence it is unfair to UNIv3 prices to adjust with this as is. 
# Moreover, where the data is available to observe on Etherscan, we did not find an open source api call that can 
# deliver this. If you know of any, please share!). 
# Hence, a manual sample was taken for UNIv3 transactions and a generous lower bound is chosen of 60%  
# Moreover another estimator is the ETH price which is chosen to be 1500. This could be improved by fetching the
# ETH price on a block by block basis. 

limit_used_gas_factor = 0.6

uni_df['percent_adjust'] = ((uni_df['gasUSD'] / uni_df['swaps_amountOutUSD'])* limit_used_gas_factor) +1

/tmp/ipykernel_37253/521485859.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_df['percent_adjust'] = ((uni_df['gasUSD'] / uni_df['swaps_amountOutUSD'])* limit_used_gas_factor) +1


In [374]:
uni_df['uni_in_out_ratio_adjusted'] = uni_df['uni_in_out_ratio'] * uni_df['percent_adjust']

/tmp/ipykernel_37253/3374149081.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uni_df['uni_in_out_ratio_adjusted'] = uni_df['uni_in_out_ratio'] * uni_df['percent_adjust']


In [375]:
uni_df.head(5)

,swaps_id,swaps_hash,swaps_protocol_id,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,tokenIn_symbol,tokenIn_decimal,swaps_amountIn,swaps_tokenOut_id,tokenOut_symbol,tokenOut_decimal,swaps_amountOut,swaps_gasLimit,swaps_gasUsed,swaps_gasPrice,swaps_pool_id,swaps_amountOutUSD,univ3_In_amount,univ3_Out_amount,uni_in_out_ratio,gas_wei,gasUSD,percent_adjust,uni_in_out_ratio_adjusted
0,0xe4fab4a0084618b492ea1c6f3cea09c7f74b63a297849af91b35b5ba481037eb37000000,0xe4fab4a0084618b492ea1c6f3cea09c7f74b63a297849af91b35b5ba481037eb,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797679,1678450691,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,2823591691214060678,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,3911206227,350770,None,23075521357,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,3911.206227,2.823592,3911.206227,0.000722,2.307552e-08,12.141301,1.001863,0.000723
1,0x330c4168f95efddde5fb82ffe1832991c88c1e20337ebac8e8cbdef8e36a27311f000000,0x330c4168f95efddde5fb82ffe1832991c88c1e20337ebac8e8cbdef8e36a2731,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797679,1678450691,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,128031297941806173106,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,177245294150,1825717,None,23237437847,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,177245.294150,128.031298,177245.29415,0.000722,2.323744e-08,63.637478,1.000215,0.000722
2,0x172c4f3a11255b98b4811c496588f3ad6b21fe11556f1e0d50e4fd50b23d5a0ef0000000,0x172c4f3a11255b98b4811c496588f3ad6b21fe11556f1e0d50e4fd50b23d5a0e,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797679,1678450691,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,956535636118829721,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,1325000000,235837,None,23075521357,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,1325.000000,0.956536,1325.0,0.000722,2.307552e-08,8.163093,1.003696,0.000725
3,0xd1e17a0b410ff5023bfae437c2ba672938e52dcebc3279fc40cc5f9b76bedeba5f000000,0xd1e17a0b410ff5023bfae437c2ba672938e52dcebc3279fc40cc5f9b76bedeba,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797468,1678448135,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,144833161400523502,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,200000000,256347,None,24828566233,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,200.000000,0.144833,200.0,0.000724,2.482857e-08,9.547093,1.028641,0.000745
4,0xdc7f74fe8ab9fda790cdf55610127f2514c362da15a231728b9a4da99ee16e8802000000,0xdc7f74fe8ab9fda790cdf55610127f2514c362da15a231728b9a4da99ee16e88,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797513,1678448675,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,57127575212311646324,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,78355027835,338535,None,26439661157,0xe0554a476a092703abdb3ef35c80e0d76d32939f,78355.027835,57.127575,78355.027835,0.000729,2.643966e-08,13.426126,1.000103,0.000729


### 4. Merge COW and  Univ3 

Notes: 

- In cow, the sell and buy are from user's point of view. In Uni, the in and out are from pool point of view. 
so sell = in and buy = out 

- since ratios defined here are all sell / buy then the higher the number the worse the price. So if COW < Uni it means the COW price was better from the user's point of view. 

In [376]:
# merge trades and swaps on timestamp value, tokens, and direction of trade. We use outer join because we want to keep all trades and swaps data and backfill swap values
cow_uni_df = pd.merge(cow_df, uni_df, left_on=['trades_timestamp','trades_sellToken_id','trades_buyToken_id'], right_on=['swaps_timestamp','swaps_tokenIn_id','swaps_tokenOut_id'])

In [377]:
# re sorting by desceding timesamps keeps things tidy
cow_uni_df.sort_values(by='trades_timestamp', ascending=False, inplace=True)

In [378]:
cow_uni_df.shape

(37, 44)

In [379]:
cow_uni_df.dtypes

trades_id                     object
trades_timestamp               int64
trades_gasPrice                int64
trades_feeAmount              object
trades_txHash                 object
trades_settlement_id          object
trades_sellAmount             object
sell_token_decimal             int64
trades_buyAmount              object
buy_token_decimal              int64
trades_sellToken_id           object
trades_buyToken_id            object
trades_order_id               object
sell_token_symbol             object
buy_token_symbol              object
trades_sellAmountUsd         float64
sell_amount                   object
buy_amount                    object
cow_sell_buy_ratio            object
swaps_id                      object
swaps_hash                    object
swaps_protocol_id             object
swaps_blockNumber              int64
swaps_timestamp                int64
swaps_tokenIn_id              object
tokenIn_symbol                object
tokenIn_decimal                int64
s

In [380]:
cow_uni_df.head(5)

,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_sellAmount,sell_token_decimal,trades_buyAmount,buy_token_decimal,trades_sellToken_id,trades_buyToken_id,trades_order_id,sell_token_symbol,buy_token_symbol,trades_sellAmountUsd,sell_amount,buy_amount,cow_sell_buy_ratio,swaps_id,swaps_hash,swaps_protocol_id,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,tokenIn_symbol,tokenIn_decimal,swaps_amountIn,swaps_tokenOut_id,tokenOut_symbol,tokenOut_decimal,swaps_amountOut,swaps_gasLimit,swaps_gasUsed,swaps_gasPrice,swaps_pool_id,swaps_amountOutUSD,univ3_In_amount,univ3_Out_amount,uni_in_out_ratio,gas_wei,gasUSD,percent_adjust,uni_in_out_ratio_adjusted
0,0x37c468b11783fb53b91a6ae3d89d65eb16272ea3b3fc014e6b099a0a468c80ff1b7a52a8902d07571d4eb862c0c9e6fa00f7459d640b2bf0|0xbef3f02b08cdeb51748bd03d6755234e53883747038b59364d7e69a2e4d2730e|9,1678451999,21348119132,16813307,0xbef3f02b08cdeb51748bd03d6755234e53883747038b59364d7e69a2e4d2730e,0xbef3f02b08cdeb51748bd03d6755234e53883747038b59364d7e69a2e4d2730e,67539828367,6,70000000000000000000000,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x3446dd70b2d52a6bf4a5a192d9b0a161295ab7f9,0x37c468b11783fb53b91a6ae3d89d65eb16272ea3b3fc014e6b099a0a468c80ff1b7a52a8902d07571d4eb862c0c9e6fa00f7459d640b2bf0,USDC,SUDO,67539.828367,67539.828367,70000.0,0.964855,0xbef3f02b08cdeb51748bd03d6755234e53883747038b59364d7e69a2e4d2730e23000000,0xbef3f02b08cdeb51748bd03d6755234e53883747038b59364d7e69a2e4d2730e,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797786,1678451999,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,1338479807,0x3446dd70b2d52a6bf4a5a192d9b0a161295ab7f9,SUDO,18,1390066832147591416448,772758,None,21348119132,0xdf709525c5880322422a10e045c1e0fa82cda1b2,1300.686993,1338.479807,1390.066832,0.962889,2.134812e-08,24.745395,1.011415,0.97388
7,0x9aaa615109ca4210b6707c287cf6d1590038953e8d37b6a1c91ba93102a7e05bf1102311370b85fca6c29011e0fcc34697228430640b2987|0xf60f33ba6ad38388320d3edfcfabc0f556dad9e89d78fbfdf725d38722aedf86|80,1678451399,24100323552,38717848,0xf60f33ba6ad38388320d3edfcfabc0f556dad9e89d78fbfdf725d38722aedf86,0xf60f33ba6ad38388320d3edfcfabc0f556dad9e89d78fbfdf725d38722aedf86,155073827174,6,70424168969713533893141,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x5a98fcbea516cf06857215779fd812ca3bef1b32,0x9aaa615109ca4210b6707c287cf6d1590038953e8d37b6a1c91ba93102a7e05bf1102311370b85fca6c29011e0fcc34697228430640b2987,USDC,LDO,155073.827174,155073.827174,70424.16897,2.201997,0xf60f33ba6ad38388320d3edfcfabc0f556dad9e89d78fbfdf725d38722aedf868f000000,0xf60f33ba6ad38388320d3edfcfabc0f556dad9e89d78fbfdf725d38722aedf86,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797737,1678451399,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,2482112100,0x5a98fcbea516cf06857215779fd812ca3bef1b32,LDO,18,1129608757546869182938,2134797,None,24100323552,0x78235d08b2ae7a3e00184329212a4d7acd2f9985,2474.416637,2482.1121,1129.608758,2.19732,2.410032e-08,77.173948,1.018713,2.238439
9,0x64b92140bea376e445bd09056b0db36ff8371ce3fb6d036da4ee45378ba6ef488331cede2c4a8e68bbef870f711eaf8169310fda640b27b3|0xec6be1609c701171520374fbe3e7ab826f491987d8fcec33d947164cd058330b|87,1678450931,22945518066,18365892818026340352,0xec6be1609c701171520374fbe3e7ab826f491987d8fcec33d947164cd058330b,0xec6be1609c701171520374fbe3e7ab826f491987d8fcec33d947164cd058330b,6001410856224179914342,18,2830750025,6,0x5283d291dbcf85356a21ba090e6db59121208b44,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x64b92140bea376e445bd09056b0db36ff8371ce3fb6d036da4ee45378ba6ef488331cede2c4a8e68bbef870f711eaf8169310fda640b27b3,BLUR,USDC,2840.402766,6001.410856,2830.750025,2.120078,0xec6be1609c701171520374fbe3e7ab826f491987d8fcec33d947164cd058330b83000000,0xec6be1609c701171520374fbe3e7ab826f491987d8fcec33d947164cd058330b,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797699,1678450931,0x5283d291dbcf85356a21ba090e6db59121208b44,BLUR,18,5983044963406153573990,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,2830750025,288446,None

###  5. Group trades in same block and derive BlockMin and VWAP  

In [381]:
# Validate grouping 
# Check that transactions are grouped by timestamp, sell token, and buy token. i.e. all transactions have the same 
# three are grouped together. 

cow_uni_df_group_test = cow_uni_df.groupby(['trades_timestamp', 'trades_sellToken_id', 'trades_buyToken_id'])

In [382]:
group_sizes = cow_uni_df_group_test.size()
print(group_sizes)

trades_timestamp  trades_sellToken_id                         trades_buyToken_id                        
1678446515        0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    8
1678446755        0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    1
1678447187        0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    2
1678447487        0x5a98fcbea516cf06857215779fd812ca3bef1b32  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    1
1678448087        0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    5
1678448135        0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2    1
1678448579        0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    4
1678449395        0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  0x5283d291dbcf85356a21ba090e6db59121208b44    1
1678449587     

Grouping works by.... 

Initially we isolate the UNI trades by slicing away just the UNI dataframe. 



In [383]:
# Split the merged cow_uni_df into just the univ3 side to allow further manipulation and grouping by block. 
uni_temp_df = cow_uni_df.loc[:, 'swaps_id':]

In [384]:
uni_temp_df.shape

(37, 25)

In [385]:
uni_temp_df.dtypes

swaps_id                      object
swaps_hash                    object
swaps_protocol_id             object
swaps_blockNumber              int64
swaps_timestamp                int64
swaps_tokenIn_id              object
tokenIn_symbol                object
tokenIn_decimal                int64
swaps_amountIn                object
swaps_tokenOut_id             object
tokenOut_symbol               object
tokenOut_decimal               int64
swaps_amountOut               object
swaps_gasLimit                 int64
swaps_gasUsed                 object
swaps_gasPrice                 int64
swaps_pool_id                 object
swaps_amountOutUSD           float64
univ3_In_amount               object
univ3_Out_amount              object
uni_in_out_ratio              object
gas_wei                      float64
gasUSD                       float64
percent_adjust               float64
uni_in_out_ratio_adjusted     object
dtype: object

In [386]:
uni_temp_df.head(5)

,swaps_id,swaps_hash,swaps_protocol_id,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,tokenIn_symbol,tokenIn_decimal,swaps_amountIn,swaps_tokenOut_id,tokenOut_symbol,tokenOut_decimal,swaps_amountOut,swaps_gasLimit,swaps_gasUsed,swaps_gasPrice,swaps_pool_id,swaps_amountOutUSD,univ3_In_amount,univ3_Out_amount,uni_in_out_ratio,gas_wei,gasUSD,percent_adjust,uni_in_out_ratio_adjusted
0,0xbef3f02b08cdeb51748bd03d6755234e53883747038b59364d7e69a2e4d2730e23000000,0xbef3f02b08cdeb51748bd03d6755234e53883747038b59364d7e69a2e4d2730e,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797786,1678451999,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,1338479807,0x3446dd70b2d52a6bf4a5a192d9b0a161295ab7f9,SUDO,18,1390066832147591416448,772758,None,21348119132,0xdf709525c5880322422a10e045c1e0fa82cda1b2,1300.686993,1338.479807,1390.066832,0.962889,2.134812e-08,24.745395,1.011415,0.97388
7,0xf60f33ba6ad38388320d3edfcfabc0f556dad9e89d78fbfdf725d38722aedf868f000000,0xf60f33ba6ad38388320d3edfcfabc0f556dad9e89d78fbfdf725d38722aedf86,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797737,1678451399,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,2482112100,0x5a98fcbea516cf06857215779fd812ca3bef1b32,LDO,18,1129608757546869182938,2134797,None,24100323552,0x78235d08b2ae7a3e00184329212a4d7acd2f9985,2474.416637,2482.1121,1129.608758,2.19732,2.410032e-08,77.173948,1.018713,2.238439
9,0xec6be1609c701171520374fbe3e7ab826f491987d8fcec33d947164cd058330b83000000,0xec6be1609c701171520374fbe3e7ab826f491987d8fcec33d947164cd058330b,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797699,1678450931,0x5283d291dbcf85356a21ba090e6db59121208b44,BLUR,18,5983044963406153573990,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,2830750025,288446,None,22945518066,0x92ab871abb9d567aa276b2ce58d0203d84e0181e,2830.750025,5983.044963,2830.750025,2.11359,2.294552e-08,9.927814,1.002104,2.118038
1,0x0aa5902035c2a63211880d37e830ff0fba56e0acd70095ce74870dcac6e92410da000000,0x0aa5902035c2a63211880d37e830ff0fba56e0acd70095ce74870dcac6e92410,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797696,1678450895,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,49991188863,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,35916707508293598938,903109,None,22470748017,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,49801.019435,49991.188863,35.916708,1391.864465,2.247075e-08,30.440302,1.000367,1392.374921
10,0xe4fab4a0084618b492ea1c6f3cea09c7f74b63a297849af91b35b5ba481037eb37000000,0xe4fab4a0084618b492ea1c6f3cea09c7f74b63a297849af91b35b5ba481037eb,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797679,1678450691,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH,18,2823591691214060678,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,3911206227,350770,None,23075521357,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,3911.206227,2.823592,3911.206227,0.000722,2.307552e-08,12.141301,1.001863,0.000723


In [387]:
# Validate grouping 

grouped_df_test = uni_temp_df.groupby(['swaps_timestamp', 'swaps_tokenIn_id', 'swaps_tokenOut_id'])
group_sizes = grouped_df_test.size()
print(group_sizes)

swaps_timestamp  swaps_tokenIn_id                            swaps_tokenOut_id                         
1678446515       0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    8
1678446755       0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    1
1678447187       0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    2
1678447487       0x5a98fcbea516cf06857215779fd812ca3bef1b32  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    1
1678448087       0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    5
1678448135       0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2    1
1678448579       0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    4
1678449395       0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2  0x5283d291dbcf85356a21ba090e6db59121208b44    1
1678449587       0xa0b86

In [388]:
# BEST price in block

In [389]:
# Group and perform MIN operator to obtain lowest value in univ3. (i.e. The minimum price in the block) 
grouped_df_min = uni_temp_df.groupby(['swaps_timestamp', 'swaps_tokenIn_id', 'swaps_tokenOut_id']).agg({'uni_in_out_ratio_adjusted': 'min'})

In [390]:
grouped_df_min.shape

(17, 1)

In [391]:
grouped_df_min.sort_values(by='swaps_timestamp', ascending=False, inplace=True)

In [392]:
grouped_df_min = grouped_df_min.rename(columns={'uni_in_out_ratio_adjusted': 'univ3BlockMin'})

In [393]:
grouped_df_min

univ3BlockMin
swaps_timestamp swaps_tokenIn_id                           swaps_tokenOut_id                                        
1678451999      0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 0x3446dd70b2d52a6bf4a5a192d9b0a161295ab7f9       0.973880
1678451399      0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 0x5a98fcbea516cf06857215779fd812ca3bef1b32       2.238439
1678450931      0x5283d291dbcf85356a21ba090e6db59121208b44 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48       2.118038
1678450895      0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2    1392.374921
1678450691      0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48       0.000722
1678450367      0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2    1387.167932
1678449647      0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48       0.000726
                                                           0xa8b919680258d369114910511cc87595aec0be6d       0.005594
1678449587      0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 0x5a98fcbea516cf06857215779fd812ca3bef1b32       2.247297
1678449395      0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0x5283d291dbcf85356a21ba090e6db59121208b44       0.000339
1678448579      0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48       0.000727
1678448135      0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2    1384.630568
1678448087      0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48       0.000726
1678447487      0x5a98fcbea516cf06857215779fd812ca3bef1b32 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48       0.475543
1678447187      0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48       0.000726
1678446755      0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48       0.000723
1678446515      0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48       0.000722

**VWAP of block**

The calculation is a simple weighted average formula. The weights used are the volume of the trade. Since trades in a group and have the same buy token and same sell token, we chose the `univ3_Out_amount` as our weights. Using the `univ3_In_amount` should have the same result. 

VWAP of blocks. 

In [394]:
uni_temp_df2 = uni_temp_df

In [395]:
uni_temp_df2['weighted_ratio'] = uni_temp_df2['univ3_Out_amount'] * uni_temp_df2['uni_in_out_ratio_adjusted']

In [396]:
uni_temp_df2.head(2)

,swaps_id,swaps_hash,swaps_protocol_id,swaps_blockNumber,swaps_timestamp,swaps_tokenIn_id,tokenIn_symbol,tokenIn_decimal,swaps_amountIn,swaps_tokenOut_id,tokenOut_symbol,tokenOut_decimal,swaps_amountOut,swaps_gasLimit,swaps_gasUsed,swaps_gasPrice,swaps_pool_id,swaps_amountOutUSD,univ3_In_amount,univ3_Out_amount,uni_in_out_ratio,gas_wei,gasUSD,percent_adjust,uni_in_out_ratio_adjusted,weighted_ratio
0,0xbef3f02b08cdeb51748bd03d6755234e53883747038b59364d7e69a2e4d2730e23000000,0xbef3f02b08cdeb51748bd03d6755234e53883747038b59364d7e69a2e4d2730e,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797786,1678451999,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,1338479807,0x3446dd70b2d52a6bf4a5a192d9b0a161295ab7f9,SUDO,18,1390066832147591416448,772758,None,21348119132,0xdf709525c5880322422a10e045c1e0fa82cda1b2,1300.686993,1338.479807,1390.066832,0.962889,2.134812e-08,24.745395,1.011415,0.97388,1353.758446
7,0xf60f33ba6ad38388320d3edfcfabc0f556dad9e89d78fbfdf725d38722aedf868f000000,0xf60f33ba6ad38388320d3edfcfabc0f556dad9e89d78fbfdf725d38722aedf86,0x1f98431c8ad98523631ae4a59f267346ea31f984,16797737,1678451399,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC,6,2482112100,0x5a98fcbea516cf06857215779fd812ca3bef1b32,LDO,18,1129608757546869182938,2134797,None,24100323552,0x78235d08b2ae7a3e00184329212a4d7acd2f9985,2474.416637,2482.1121,1129.608758,2.19732,2.410032e-08,77.173948,1.018713,2.238439,2528.560476


In [397]:
# chose weights as swap amounts out as they include all data in integers. 
grouped_df_vwap = uni_temp_df.groupby(['swaps_timestamp', 'swaps_tokenIn_id', 'swaps_tokenOut_id']).agg({'univ3_Out_amount':'sum','weighted_ratio':'sum'})

In [398]:
grouped_df_vwap['univ3_vwap'] = grouped_df_vwap['weighted_ratio'] / grouped_df_vwap['univ3_Out_amount'] 

In [399]:
grouped_df_vwap.sort_values(by='swaps_timestamp', ascending=False, inplace=True)

In [400]:
grouped_df_vwap.shape

(17, 3)

In [401]:
grouped_df_vwap

univ3_Out_amount weighted_ratio   univ3_vwap
swaps_timestamp swaps_tokenIn_id                           swaps_tokenOut_id                                                                      
1678451999      0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 0x3446dd70b2d52a6bf4a5a192d9b0a161295ab7f9      1390.066832    1353.758446      0.97388
1678451399      0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 0x5a98fcbea516cf06857215779fd812ca3bef1b32      1129.608758    2528.560476     2.238439
1678450931      0x5283d291dbcf85356a21ba090e6db59121208b44 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48      2830.750025    5995.634961     2.118038
1678450895      0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2        35.916708   50009.522788  1392.374921
1678450691      0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    182481.500377     131.847801     0.000723
1678450367      0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2        20.201222   28076.229933  1389.828322
1678449647      0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48      4320.082087       3.138061     0.000726
                                                           0xa8b919680258d369114910511cc87595aec0be6d        337.90838        1.89033     0.005594
1678449587      0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 0x5a98fcbea516cf06857215779fd812ca3bef1b32       711.135801    1598.133527     2.247297
1678449395      0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0x5283d291dbcf85356a21ba090e6db59121208b44     16405.704162       5.568311     0.000339
1678448579      0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    104172.592736      75.834551     0.000728
1678448135      0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48 0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2         7.229934    10010.78767  1384.630568
1678448087      0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48     86605.050622      62.918622     0.000727
1678447487      0x5a98fcbea516cf06857215779fd812ca3bef1b32 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48      2612.021973    1242.128582     0.475543
1678447187      0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48     44450.900767      32.274149     0.000726
1678446755      0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48    173461.007554     125.409521     0.000723
1678446515      0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48      66622.62057      48.162642     0.000723

#### Step 3: Merge cow trades with univ3_blockMax and univ3_vwap 

In [402]:
# Perform the merge to include vwap result
cow_univ3_merge1_df = pd.merge(cow_df, grouped_df_min, left_on=['trades_timestamp','trades_sellToken_id','trades_buyToken_id'], right_on=['swaps_timestamp','swaps_tokenIn_id','swaps_tokenOut_id'])

In [403]:
cow_univ3_merge1_df.sort_values(by='trades_timestamp', ascending=False, inplace=True)

In [404]:
cow_univ3_merge1_df.shape

(17, 20)

In [405]:
cow_univ3_merge1_df

,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_sellAmount,sell_token_decimal,trades_buyAmount,buy_token_decimal,trades_sellToken_id,trades_buyToken_id,trades_order_id,sell_token_symbol,buy_token_symbol,trades_sellAmountUsd,sell_amount,buy_amount,cow_sell_buy_ratio,univ3BlockMin
0,0x37c468b11783fb53b91a6ae3d89d65eb16272ea3b3fc014e6b099a0a468c80ff1b7a52a8902d07571d4eb862c0c9e6fa00f7459d640b2bf0|0xbef3f02b08cdeb51748bd03d6755234e53883747038b59364d7e69a2e4d2730e|9,1678451999,21348119132,16813307,0xbef3f02b08cdeb51748bd03d6755234e53883747038b59364d7e69a2e4d2730e,0xbef3f02b08cdeb51748bd03d6755234e53883747038b59364d7e69a2e4d2730e,67539828367,6,70000000000000000000000,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x3446dd70b2d52a6bf4a5a192d9b0a161295ab7f9,0x37c468b11783fb53b91a6ae3d89d65eb16272ea3b3fc014e6b099a0a468c80ff1b7a52a8902d07571d4eb862c0c9e6fa00f7459d640b2bf0,USDC,SUDO,67539.828367,67539.828367,70000.0,0.964855,0.973880
4,0x9aaa615109ca4210b6707c287cf6d1590038953e8d37b6a1c91ba93102a7e05bf1102311370b85fca6c29011e0fcc34697228430640b2987|0xf60f33ba6ad38388320d3edfcfabc0f556dad9e89d78fbfdf725d38722aedf86|80,1678451399,24100323552,38717848,0xf60f33ba6ad38388320d3edfcfabc0f556dad9e89d78fbfdf725d38722aedf86,0xf60f33ba6ad38388320d3edfcfabc0f556dad9e89d78fbfdf725d38722aedf86,155073827174,6,70424168969713533893141,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x5a98fcbea516cf06857215779fd812ca3bef1b32,0x9aaa615109ca4210b6707c287cf6d1590038953e8d37b6a1c91ba93102a7e05bf1102311370b85fca6c29011e0fcc34697228430640b2987,USDC,LDO,155073.827174,155073.827174,70424.16897,2.201997,2.238439
6,0x64b92140bea376e445bd09056b0db36ff8371ce3fb6d036da4ee45378ba6ef488331cede2c4a8e68bbef870f711eaf8169310fda640b27b3|0xec6be1609c701171520374fbe3e7ab826f491987d8fcec33d947164cd058330b|87,1678450931,22945518066,18365892818026340352,0xec6be1609c701171520374fbe3e7ab826f491987d8fcec33d947164cd058330b,0xec6be1609c701171520374fbe3e7ab826f491987d8fcec33d947164cd058330b,6001410856224179914342,18,2830750025,6,0x5283d291dbcf85356a21ba090e6db59121208b44,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x64b92140bea376e445bd09056b0db36ff8371ce3fb6d036da4ee45378ba6ef488331cede2c4a8e68bbef870f711eaf8169310fda640b27b3,BLUR,USDC,2840.402766,6001.410856,2830.750025,2.120078,2.118038
1,0x7d09d0b228748869369b0e2cde9b1ff60d3aa6c9635cf5a7e550558a95dc5479c8aec5c402386ccacbcaab0a39c0f90ed3dbf645640b25c1|0x0aa5902035c2a63211880d37e830ff0fba56e0acd70095ce74870dcac6e92410|120,1678450895,22470748017,8811137,0x0aa5902035c2a63211880d37e830ff0fba56e0acd70095ce74870dcac6e92410,0x0aa5902035c2a63211880d37e830ff0fba56e0acd70095ce74870dcac6e92410,50000000000,6,35918539681856220722,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x7d09d0b228748869369b0e2cde9b1ff60d3aa6c9635cf5a7e550558a95dc5479c8aec5c402386ccacbcaab0a39c0f90ed3dbf645640b25c1,USDC,WETH,50000.000000,50000.0,35.91854,1392.038776,1392.374921
7,0x57f44a21f72e6744527c02870b5e6610e2693faf5258f84d5a1613e417ee28ee8f02fbb56c318814de0e96bc83bb50fd3340a7eb640b2608|0x330c4168f95efddde5fb82ffe1832991c88c1e20337ebac8e8cbdef8e36a2731|0,1678450691,23237437847,40689992411037368,0x330c4168f95efddde5fb82ffe1832991c88c1e20337ebac8e8cbdef8e36a2731,0x330c4168f95efddde5fb82ffe1832991c88c1e20337ebac8e8cbdef8e36a2731,537987320000000000000,18,744593980207,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x57f44a21f72e6744527c02870b5e6610e2693faf5258f84d5a1613e417ee28ee8f02fbb56c318814de0e96bc83bb50fd3340a7eb640b2608,WETH,USDC,744236.683370,537.98732,744593.980207,0.000723,0.000722
2,0xa94ce10368e99719a54cc734d972ce41a0f90b570f902ed849af267edda79ece7f9c8843926de4b2ea6cbf9381f9f2fa81c58ed6640b2559|0xca75e37b9d103a446274e1a2c6fe831bd326f3c79b78407f9e2b237cc82494ec|208,1678450367,28056588752,6449809,0xca75e37b9d103a446274e1a2c6fe831bd326f3c79b78407f9e2b237cc82494ec,0xca75e37b9d103a446274e1a2c6fe831bd326f3c79b78407f9e2b237cc82494ec,

In [406]:
# Perform the merge to include vwap result
cow_univ3_merge2_df = pd.merge(cow_univ3_merge1_df, grouped_df_vwap, 
                               left_on=['trades_timestamp', 'trades_sellToken_id', 'trades_buyToken_id'], 
                               right_on=['swaps_timestamp', 'swaps_tokenIn_id', 'swaps_tokenOut_id'], 
                               how='left')


In [407]:
cow_univ3_merge2_df.head(5)

,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_sellAmount,sell_token_decimal,trades_buyAmount,buy_token_decimal,trades_sellToken_id,trades_buyToken_id,trades_order_id,sell_token_symbol,buy_token_symbol,trades_sellAmountUsd,sell_amount,buy_amount,cow_sell_buy_ratio,univ3BlockMin,univ3_Out_amount,weighted_ratio,univ3_vwap
0,0x37c468b11783fb53b91a6ae3d89d65eb16272ea3b3fc014e6b099a0a468c80ff1b7a52a8902d07571d4eb862c0c9e6fa00f7459d640b2bf0|0xbef3f02b08cdeb51748bd03d6755234e53883747038b59364d7e69a2e4d2730e|9,1678451999,21348119132,16813307,0xbef3f02b08cdeb51748bd03d6755234e53883747038b59364d7e69a2e4d2730e,0xbef3f02b08cdeb51748bd03d6755234e53883747038b59364d7e69a2e4d2730e,67539828367,6,70000000000000000000000,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x3446dd70b2d52a6bf4a5a192d9b0a161295ab7f9,0x37c468b11783fb53b91a6ae3d89d65eb16272ea3b3fc014e6b099a0a468c80ff1b7a52a8902d07571d4eb862c0c9e6fa00f7459d640b2bf0,USDC,SUDO,67539.828367,67539.828367,70000.0,0.964855,0.973880,1390.066832,1353.758446,0.97388
1,0x9aaa615109ca4210b6707c287cf6d1590038953e8d37b6a1c91ba93102a7e05bf1102311370b85fca6c29011e0fcc34697228430640b2987|0xf60f33ba6ad38388320d3edfcfabc0f556dad9e89d78fbfdf725d38722aedf86|80,1678451399,24100323552,38717848,0xf60f33ba6ad38388320d3edfcfabc0f556dad9e89d78fbfdf725d38722aedf86,0xf60f33ba6ad38388320d3edfcfabc0f556dad9e89d78fbfdf725d38722aedf86,155073827174,6,70424168969713533893141,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x5a98fcbea516cf06857215779fd812ca3bef1b32,0x9aaa615109ca4210b6707c287cf6d1590038953e8d37b6a1c91ba93102a7e05bf1102311370b85fca6c29011e0fcc34697228430640b2987,USDC,LDO,155073.827174,155073.827174,70424.16897,2.201997,2.238439,1129.608758,2528.560476,2.238439
2,0x64b92140bea376e445bd09056b0db36ff8371ce3fb6d036da4ee45378ba6ef488331cede2c4a8e68bbef870f711eaf8169310fda640b27b3|0xec6be1609c701171520374fbe3e7ab826f491987d8fcec33d947164cd058330b|87,1678450931,22945518066,18365892818026340352,0xec6be1609c701171520374fbe3e7ab826f491987d8fcec33d947164cd058330b,0xec6be1609c701171520374fbe3e7ab826f491987d8fcec33d947164cd058330b,6001410856224179914342,18,2830750025,6,0x5283d291dbcf85356a21ba090e6db59121208b44,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x64b92140bea376e445bd09056b0db36ff8371ce3fb6d036da4ee45378ba6ef488331cede2c4a8e68bbef870f711eaf8169310fda640b27b3,BLUR,USDC,2840.402766,6001.410856,2830.750025,2.120078,2.118038,2830.750025,5995.634961,2.118038
3,0x7d09d0b228748869369b0e2cde9b1ff60d3aa6c9635cf5a7e550558a95dc5479c8aec5c402386ccacbcaab0a39c0f90ed3dbf645640b25c1|0x0aa5902035c2a63211880d37e830ff0fba56e0acd70095ce74870dcac6e92410|120,1678450895,22470748017,8811137,0x0aa5902035c2a63211880d37e830ff0fba56e0acd70095ce74870dcac6e92410,0x0aa5902035c2a63211880d37e830ff0fba56e0acd70095ce74870dcac6e92410,50000000000,6,35918539681856220722,18,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x7d09d0b228748869369b0e2cde9b1ff60d3aa6c9635cf5a7e550558a95dc5479c8aec5c402386ccacbcaab0a39c0f90ed3dbf645640b25c1,USDC,WETH,50000.000000,50000.0,35.91854,1392.038776,1392.374921,35.916708,50009.522788,1392.374921
4,0x57f44a21f72e6744527c02870b5e6610e2693faf5258f84d5a1613e417ee28ee8f02fbb56c318814de0e96bc83bb50fd3340a7eb640b2608|0x330c4168f95efddde5fb82ffe1832991c88c1e20337ebac8e8cbdef8e36a2731|0,1678450691,23237437847,40689992411037368,0x330c4168f95efddde5fb82ffe1832991c88c1e20337ebac8e8cbdef8e36a2731,0x330c4168f95efddde5fb82ffe1832991c88c1e20337ebac8e8cbdef8e36a2731,537987320000000000000,18,744593980207,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x57f44a21f72e6744527c02870b5e6610e2693faf5258f84d5a1613e417ee28ee8f02fbb56c318814de0e96bc83bb50fd3340a7eb640b2608,WETH,USDC,744236.683370,537.98732,744593.980207,0.000723,0.000722,182481.500377,131.847801,0.000723


In [408]:
cow_univ3_merge2_df.shape

(17, 23)

In [409]:
cow_univ3_merge2_df.dtypes

trades_id                object
trades_timestamp          int64
trades_gasPrice           int64
trades_feeAmount         object
trades_txHash            object
trades_settlement_id     object
trades_sellAmount        object
sell_token_decimal        int64
trades_buyAmount         object
buy_token_decimal         int64
trades_sellToken_id      object
trades_buyToken_id       object
trades_order_id          object
sell_token_symbol        object
buy_token_symbol         object
trades_sellAmountUsd    float64
sell_amount              object
buy_amount               object
cow_sell_buy_ratio       object
univ3BlockMin           float64
univ3_Out_amount         object
weighted_ratio           object
univ3_vwap               object
dtype: object

In [410]:
unique_cow_txns = len(cow_univ3_merge2_df['trades_id'].unique())
unique_cow_txns

17

In [411]:
cow_univ3_merge2_df.to_csv('zero_arb_cow_univ3_raw_data.csv')

### 6. Analysis 

Look at:
how many times cow beat maxBlock pricing. 
How many times cow beat vwap pricing 
What is sd of differences with vwap? 
For the times COW beat pricing, what tokens? 
by How much? 
group by token directions and see how stats differ. 
by How much? 
differences stats


Since we have defined ratios to be sell/ price then a higher number = worse pricing. 

In [412]:
prices_df = cow_univ3_merge2_df[['trades_sellAmountUsd','cow_sell_buy_ratio', 'univ3BlockMin', 'univ3_vwap']]

In [413]:
prices_df

,trades_sellAmountUsd,cow_sell_buy_ratio,univ3BlockMin,univ3_vwap
0,67539.828367,0.964855,0.973880,0.97388
1,155073.827174,2.201997,2.238439,2.238439
2,2840.402766,2.120078,2.118038,2.118038
3,50000.000000,1392.038776,1392.374921,1392.374921
4,744236.683370,0.000723,0.000722,0.000723
5,20000.000000,1386.193545,1387.167932,1389.828322
6,6916.860823,0.005557,0.005594,0.005594
7,69860.294310,0.000722,0.000726,0.000726
8,127450.000000,2.183313,2.247297,2.247297
9,17845.500923,0.000339,0.000339,0.000339


In [414]:
# calculate percentage difference 
prices_df['cow_blockMax_diff'] = ((prices_df['cow_sell_buy_ratio'] - prices_df['univ3BlockMin']) / prices_df['uni_in_out_ratio_adjusted_blockMax']) * 100

KeyError: 'uni_in_out_ratio_adjusted_blockMax'

In [415]:
# calculate percentage difference 
prices_df['cow_vwap_diff'] = ((prices_df['cow_sell_buy_ratio'] - prices_df['univ3_vwap']) / prices_df['univ3_vwap']) * 100

/tmp/ipykernel_37253/3309710702.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices_df['cow_vwap_diff'] = ((prices_df['cow_sell_buy_ratio'] - prices_df['univ3_vwap']) / prices_df['univ3_vwap']) * 100


In [ ]:
prices_df

In [ ]:
# Comparison with blockMax 

In [ ]:
# Count the number of values less than 0
blockMax_lt_zero = (prices_df['cow_blockMax_diff'] < 0).sum()

# Count the number of values greater than 0
blockMax_gt_zero = (prices_df['cow_blockMax_diff'] > 0).sum()

# Count the number of values equal to 0
blockMax_eq_zero = (prices_df['cow_blockMax_diff'] == 0).sum()


total_count = len(prices_df)
blockMax_gt_zero_pct = blockMax_gt_zero / total_count * 100
blockMax_eq_zero_pct = blockMax_eq_zero/ total_count * 100
blockMax_lt_zero_pct = blockMax_lt_zero/ total_count * 100



print("Number of values < 0:", blockMax_lt_zero, '/', blockMax_lt_zero_pct, '%')
print("Number of values > 0:", blockMax_gt_zero, '/', blockMax_gt_zero_pct, '%')
print("Number of values = 0:", blockMax_eq_zero, '/', blockMax_eq_zero_pct, '%')

In [ ]:
# Compute the average of values greater than 0
blockMax_gt_zero_mean = prices_df.loc[prices_df['cow_blockMax_diff'] > 0, 'cow_blockMax_diff'].mean()

# Compute the average of values less than 0
blockMax_lt_zero_mean = prices_df.loc[prices_df['cow_blockMax_diff'] < 0, 'cow_blockMax_diff'].mean()

print("Average of values > 0:", blockMax_gt_zero_mean)
print("Average of values < 0:", blockMax_lt_zero_mean)



In [ ]:
# Comparison with VWAP 

In [ ]:
# Count the number of values less than 0
VWAP_lt_zero = (prices_df['cow_vwap_diff'] < 0).sum()

# Count the number of values greater than 0
VWAP_gt_zero = (prices_df['cow_vwap_diff'] > 0).sum()

# Count the number of values equal to 0
VWAP_eq_zero = (prices_df['cow_vwap_diff'] == 0).sum()


total_count = len(prices_df)
VWAP_gt_zero_pct = VWAP_gt_zero / total_count * 100
VWAP_eq_zero_pct = VWAP_eq_zero/ total_count * 100
VWAP_lt_zero_pct = VWAP_lt_zero/ total_count * 100




print("Number of values < 0:", VWAP_lt_zero, '/', VWAP_lt_zero_pct, '%')
print("Number of values > 0:", VWAP_gt_zero, '/', VWAP_gt_zero_pct, '%')
print("Number of values = 0:", VWAP_eq_zero, '/', VWAP_eq_zero_pct, '%')

In [ ]:
# Compute the average of values greater than 0
VWAP_gt_zero_mean = prices_df.loc[prices_df['cow_vwap_diff'] > 0, 'cow_vwap_diff'].mean()

# Compute the average of values less than 0
VWAP_lt_zero_mean = prices_df.loc[prices_df['cow_vwap_diff'] < 0, 'cow_vwap_diff'].mean()

print("Average of values > 0:", VWAP_gt_zero_mean)
print("Average of values < 0:", VWAP_lt_zero_mean)